# [Will You Top the Leaderboard?](https://thefiddler.substack.com/p/will-you-top-the-leaderboard)
## October 11 2024

## Problem

_You’re doing a 30-minute workout on your stationary bike. There’s a live leaderboard that tracks your progress, along with the progress of everyone else who is currently riding, measured in units of energy called kilojoules. (For reference, one kilojoule is 1000 Watt-seconds.) Once someone completes their ride, they are removed from the leaderboard._

_Suppose many riders are doing the 30-minute workout right now, and that they all begin at random times, with many starting before you and many starting after. Further suppose that they are burning kilojoules at different constant rates (i.e., everyone is riding at constant power) that are uniformly distributed between 0 and 200 Watts._

_Halfway through (i.e., 15 minutes into) your workout, you notice that you’re exactly halfway up the leaderboard. How far up the leaderboard can you expect to be as you’re finishing your workout?_

_As an added bonus problem (though not quite Extra Credit), what’s the highest up the leaderboard you could expect to be 15 minutes into your workout?_

## Solution

A couple of things to notice about the problem desctiption:

1. We're essentially comparing the energy expended by all active participants at a given time
2. It says 'many rides' which I presume means an essentially infinite number of riders

Since we have an essentially infinite number of riders, being in the middle of the leaderboard means that we are directly in the middle of the distribution of energy expended by the typical rider which ammounts to when the CDF = 0.5.

$ E_i = P_i T_i $ Where $ P_i $ is the power and $ T_i $ is the time a rider has been riding. We know $ P_i $ is uniformly distributed between 0 and 200 Watts and $ T_i $ is uniformly distributed between 0 and 30 minutes.

To calculate the pdf of $ E_i $ we need to calculate the pdf of $ P_i $ and $ T_i $ and then convolve them.

The pdf of $ P_i $ is $ f_{P_i}(p) = \frac{1}{200} $ for $ 0 \leq p \leq 200 $ and 0 otherwise.

The pdf of $ T_i $ is $ f_{T_i}(t) = \frac{1}{30} $ for $ 0 \leq t \leq 30 $ and 0 otherwise.

The pdf of $ E_i $ is $ \displaystyle f_{E_i}(e) = \int_{-\infty}^{\infty} \left| \frac{1}{t} \right| f_{P_i}(e/t) f_{T_i}(t) dt $

Let's consider the bounds here. $ f_{P_i}(e/t) $ is non-zero only when $ e/t \leq 200 \rightarrow \frac{e}{200} \leq t $. $ f_{T_i}(t) $ is non-zero only when $ 0 \leq t \leq 30 $. Combining these we get $ \frac{e}{200} \leq t \leq 30 $.

$ \displaystyle f_{E_i}(e) = \frac{1}{30} \int_{\frac{e}{200}}^{30} \frac{1}{t} \frac{1}{200} dt = \frac{1}{6000} \int_{\frac{e}{200}}^{30} \frac{1}{t} dt = \frac{1}{6000} \ln(t) \Big|_{\frac{e}{200}}^{30} = \frac{1}{6000} \ln(30) - \frac{1}{6000} \ln\left(\frac{e}{200}\right) = \frac{1}{6000} \ln\left(\frac{30}{\frac{e}{200}}\right) = \frac{1}{6000} \ln\left(\frac{6000}{e}\right) $ where $ 0 \leq e \leq 6000 $.

Now we find where the value $ P $ that, after 15 minutes, will put you in the middle of the leaderboard. This occurs when the CDF of $ E $ is 0.5.

$ \displaystyle \int f_{E_i}(e) de = \int \frac{1}{6000} \ln\left(\frac{6000}{e}\right) de = \frac{1}{6000} \left( e \ln\left(\frac{6000}{e}\right) + e \right) = 0.5$.

Let's find this numerically

In [14]:
import math

def f(z):
    """Function whose root we want to find: f(z) = z*(ln(6000/z) + 1) - 3000"""
    if z <= 0:
        # To prevent log(0) or log of negative numbers
        return float('inf')
    return z * (math.log(6000 / z) + 1) - 3000

def f_prime(z):
    """Derivative of the function: f'(z) = ln(6000/z)"""
    if z <= 0:
        # Derivative undefined for z <= 0
        return float('inf')
    return math.log(6000 / z)

def newton_raphson(initial_guess, tol=1e-6, max_iter=100):
    """Newton-Raphson method to find root of f(z) = 0"""
    z = initial_guess
    for iteration in range(1, max_iter + 1):
        func_value = f(z)
        derivative = f_prime(z)
        
        if derivative == 0:
            print(f"Iteration {iteration}: Derivative is zero. Stopping iteration.")
            return None
        
        # Update rule
        z_new = z - func_value / derivative
        
        # Safeguard: Ensure z_new stays within (z_min, z_max)
        z_min = 1  # To prevent log(0) or negative z
        z_max = 6000  # Based on problem constraints
        if z_new < z_min:
            z_new = z_min
        elif z_new > z_max:
            z_new = z_max
        
        # Debug statements
        print(f"Iteration {iteration}: z = {z:.6f}, f(z) = {func_value:.6f}, f'(z) = {derivative:.6f}, z_new = {z_new:.6f}")
        
        # Check for convergence
        if abs(z_new - z) < tol:
            print(f"Converged to z = {z_new:.6f} after {iteration} iterations.")
            return z_new
        
        z = z_new
    
    print("Maximum iterations reached without convergence.")
    return z

# Initial guess based on prior solution (e.g., z = 1120)
initial_guess = 2000  # A reasonable starting point
median_energy = newton_raphson(initial_guess)

print(f"\nMedian Energy (z_0.5): {median_energy:.2f} kJ")
print(f"P = {median_energy/15}")


Iteration 1: z = 2000.000000, f(z) = 1197.224577, f'(z) = 1.098612, z_new = 910.239227
Iteration 2: z = 910.239227, f(z) = -373.224998, f'(z) = 1.885807, z_new = 1108.151812
Iteration 3: z = 1108.151812, f(z) = -20.106777, f'(z) = 1.689066, z_new = 1120.055893
Iteration 4: z = 1120.055893, f(z) = -0.063711, f'(z) = 1.678381, z_new = 1120.093853
Iteration 5: z = 1120.093853, f(z) = -0.000001, f'(z) = 1.678347, z_new = 1120.093853
Converged to z = 1120.093853 after 5 iterations.

Median Energy (z_0.5): 1120.09 kJ
P = 74.6729235403348


Now, we need to find your position on the leaderboard after 30 minutes knowing you have $ P=74.67 $. So we need to find the relative number of people who have a lower energy expenditure than you. This is the CDF of $ E $ evaluated at $ P \cdot 30 = 2240 $.

0.7412

$ \displaystyle \int_{0}^{2240} f_{E_i}(e) de = \int_{0}^{2240} \frac{1}{6000} \ln\left(\frac{6000}{e}\right) de = \frac{1}{6000} \left( e \ln\left(\frac{6000}{e}\right) + e \right) \Big|_{0}^{2240} = .7412 $. 

For the highest position you could have been in 15 minutes, we assume $ P = 200 $ Watts and $ E = 200 \cdot 15 = 3000 $ so we need to find the CDF of $ E $ evaluated at 3000.

$ \displaystyle \frac{1}{6000} \left( e \ln\left(\frac{6000}{e}\right) + e \right) \Big|_{0}^{3000} = .8466 $.

## Extra Credit

_Again, suppose there are many riders starting their 30-minute workouts at random times, and that their powers are uniformly distributed between 0 and 200 Watts. Now, suppose you decide that you too will be pedaling with a random (but constant) power between 0 and 200 Watts._

_If you look down at the leaderboard at a random time during this random workout, how far up the leaderboard can you expect to be, on average?_

## Solution

I'm stuggling to see how this questions isn't trivially $ 0.5 $ by some basic symmetry arguments. Let's start with a simulation to see if this is true or not.

In [32]:
# Simulate many riders and your relative rank
import numpy as np
# np.random.seed(0)

for _ in range(20):
    n_riders = 100000
    # Randomly sample the power
    powers = np.random.uniform(0, 6000, n_riders)
    # Randomly sample the amount of time spent by other riders
    times = np.random.uniform(0, 30, n_riders)

    # Sample your own power and time many times to estimate your relative rank
    n_samples = 10000
    my_powers = np.random.uniform(0, 6000, n_samples)
    my_times = np.random.uniform(0, 30, n_samples)

    # Calculate the energy for each rider
    energies = powers * times
    # Calculate the energy for each rider
    my_energies = my_powers * my_times

    ranks = []
    for my_energy in my_energies:
        # Calculate the number of riders who have lower energy
        n_lower = np.sum(energies < my_energy)
        ranks.append(n_lower/n_riders)
    print(f"Your relative rank is {np.mean(ranks):.2%}")

Your relative rank is 49.84%
Your relative rank is 50.07%
Your relative rank is 49.78%
Your relative rank is 50.08%
Your relative rank is 50.37%
Your relative rank is 49.67%
Your relative rank is 50.19%
Your relative rank is 49.82%
Your relative rank is 49.79%
Your relative rank is 49.87%
Your relative rank is 50.60%
Your relative rank is 50.24%
Your relative rank is 50.43%
Your relative rank is 50.03%
Your relative rank is 49.93%
Your relative rank is 49.75%
Your relative rank is 49.57%
Your relative rank is 50.17%
Your relative rank is 49.83%
Your relative rank is 49.84%
